In [60]:
import timeit
import pandas as pd
import os
import re
import urllib.request
import bs4

ROOTDIR = os.path.normpath("N:\_archive\\test\\trans\soundgasmNET")
df = pd.read_csv(os.path.join(ROOTDIR, "sgasm_rip_db.csv"), sep=";", encoding="utf-8", index_col=0)
df_grp = df.groupby("sgasm_user")

url = "https://soundgasm.net/u/belle_in_the_woods/F4M-The-Take-FdomJOIcountdownteasingwhispers"

In [3]:
%timeit url in df_grp.get_group("belle_in_the_woods")["URLsg"]

The slowest run took 7.66 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 527 µs per loop


In [4]:
%timeit url in df[df["sgasm_user"] == "belle_in_the_woods"]["URLsg"]

100 loops, best of 3: 2.24 ms per loop


In [5]:
def t():
    df.query("sgasm_user == 'belle_in_the_woods'")["URLsg"].isin([url]).any()

In [6]:
%timeit t()

100 loops, best of 3: 2.65 ms per loop


In [7]:
%timeit url in df["URLsg"]

The slowest run took 39.70 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 9.47 µs per loop


In [2]:
urls = ["https://soundgasm.net/u/belle_in_the_woods/F4M-The-Take-FdomJOIcountdownteasingwhispers",
       "https://soundgasm.net/u/belle_in_the_woods/F4F-Should-I-Call-intimateerotic-poetry",
       "https://soundgasm.net/u/belle_in_the_woods/F4M-In-Darkness-FsubErotic-Poetry",
       "https://soundgasm.net/u/belle_in_the_woods/F4M-Cum-Lust-cock-worshiperotic-poetry",
       "https://soundgasm.net/u/belle_in_the_woods/F4M-The-Take-FdomJOIcountdownteasingwhispers"]

def filter_alrdy_downloaded(df, dl_dict, currentusr=None):
    # OLD when passing 2pair tuples, unpack tuples in dl_list into two lists
    # url_list, title = zip(*dl_list)
    # filter dupes
    unique_urls = set(dl_dict)
    if currentusr:
        try:
            duplicate = unique_urls.intersection(df_grp.get_group(currentusr)["URLsg"].values)
        except KeyError:
            logger.info("User '{}' not yet in databas!".format(currentusr))
            duplicate = set()
    else:
        # timeit 1000: 0.19
        duplicate = unique_urls.intersection(df["URLsg"].values)

    # set.symmetric_difference()
    # Return a new set with elements in either the set or other but not both.
    # -> duplicates will get removed from unique_urls
    result = list(unique_urls.symmetric_difference(duplicate))

    return result

In [15]:
%timeit filter_alrdy_downloaded(df, urls)

1000 loops, best of 3: 705 µs per loop


In [14]:
%timeit filter_alrdy_downloaded(df, urls, currentusr="belle_in_the_woods")

1000 loops, best of 3: 488 µs per loop


In [12]:
%timeit url.split("/u/", 1)[1].split("/", 1)[0]

The slowest run took 4.19 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.63 µs per loop


In [13]:
%timeit re.search("/u/(.+)/", url).group(1)

100000 loops, best of 3: 3.28 µs per loop


In [14]:
usr = re.compile("/u/(.+)/")

In [15]:
%timeit re.search(usr, url).group(1)

100000 loops, best of 3: 3.92 µs per loop


In [87]:
def sgasm_inf_split(url):
    site = urllib.request.urlopen(url)
    html = site.read().decode('utf-8')
    site.close()
    nhtml = html.split("aria-label=\"title\">")
    title = nhtml[1].split("</div>", 1)[0]
    descript = nhtml[1].split("<div class=\"jp-description\">\n          <p style=\"white-space: pre-wrap;\">")[1].split("</p>\r\n", 1)[0]
    urlm4a = nhtml[1].split("m4a: \"")[1].split("\"\r\n", 1)[0]
    

def sgasm_re(url):
    site = urllib.request.urlopen(url)
    html = site.read().decode('utf-8')
    site.close()
    
    title = re.search("class=\"jp-title\" aria-label=\"title\">(.+)<\/div>", html).group(1)
    descript = re.search("class=\"jp-description\">\s+<p.+\">(.+)</p>", html).group(1)
    urlm4a = re.search("m4a: \"(.+)\"", html).group(1)
    
    
t_re = re.compile("class=\"jp-title\" aria-label=\"title\">(.+)<\/div>")
d_re = re.compile("class=\"jp-description\">\s+<p.+\">(.+)</p>")
url_re = re.compile("m4a: \"(.+)\"")


def sgasm_re_comp(url):
    site = urllib.request.urlopen(url)
    html = site.read().decode('utf-8')
    site.close()
    
    title = re.search(t_re, html).group(1)
    descript = re.search(d_re, html).group(1)
    urlm4a = re.search(url_re, html).group(1)
    
def sgasm_bs4(url):
    site = urllib.request.urlopen(url)
    html = site.read().decode('utf-8')
    site.close()
    soup = bs4.BeautifulSoup(html, "html.parser")
    
    title = soup.select_one("div.jp-title").text
    descr = soup.select_one("div.jp-description > p").text
    urlm4a = re.search("m4a: \"(.+)\"", html).group(1)

In [33]:
%timeit sgasm_inf_split("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_file-site.html")

1000 loops, best of 3: 427 µs per loop


In [34]:
%timeit sgasm_re("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_file-site.html")

The slowest run took 8.02 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 436 µs per loop


In [81]:
%timeit sgasm_re_comp("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_file-site.html")

1000 loops, best of 3: 417 µs per loop


In [88]:
%timeit sgasm_bs4("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_file-site.html")

100 loops, best of 3: 5.71 ms per loop


In [75]:
def rip_usr_links(sgasm_usr_url):
    site = urllib.request.urlopen(sgasm_usr_url)
    html = site.read().decode('utf-8')
    site.close()
    # links zu den einzelnen posts isolieren
    nhtml = html.split("<div class=\"sound-details\"><a href=\"")
    del nhtml[0]
    user_files = []
    for splits in nhtml:
        # teil str in form von https://soundgasm.net/u/USERNAME/link-to-post> an ">" und schreibt
        # den ersten teil in die variable url
        url = splits.split("\">", 1)[0]
        # url in die liste anfuegen
        user_files.append(url)
    return user_files


def rip_usr_links_re(sgasm_usr_url):
    site = urllib.request.urlopen(sgasm_usr_url)
    html = site.read().decode('utf-8')
    site.close()
    # links zu den einzelnen posts isolieren
    
    # .+ match one or more of any char except newline, .+? -> ?: match as few as possible
    # findall returns list of strings or list of groups -> more than 1 grp -> tuples 
    # matched = re.findall("<div class=\"sound-details\"><a href=\"(.+?)\">(.+?)<\/a>", html)
    urls = re.findall("<div class=\"sound-details\"><a href=\"(.+?)\">", html)
    # unpack list of tuples
    # url, titles = zip(*matched)
    #print(url)
    return urls


def rip_usr_links_bs4(u_url):
    site = urllib.request.urlopen(u_url)
    html = site.read().decode('utf-8')
    site.close()
    soup = bs4.BeautifulSoup(html, 'html.parser')
    
    anchs = soup.select("div.sound-details > a")
    urls = [a["href"] for a in anchs]
    return urls
        

In [52]:
%timeit rip_usr_links("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_user-site.html")

1000 loops, best of 3: 874 µs per loop


In [53]:
%timeit rip_usr_links_re("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_user-site.html")

1000 loops, best of 3: 1.49 ms per loop


In [77]:
%timeit rip_usr_links_bs4("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_user-site.html")

10 loops, best of 3: 84.3 ms per loop
